In [124]:
import numpy as np
import pandas as pd
import multiprocessing as mp
import os
from datetime import datetime, timedelta
import re
import time
import plotly.express as px

from joblib import Parallel, delayed

In [125]:
# csl_values=[0,5,10,15,20]
# isl_values=[30,35,40,50]
# threshold_values=[10,15,20,25]
csl_values=[2.5,5,7.5,10]
isl_values=[1]
threshold_values=[10,20,30]
percents=[150]
iterations=[30,40,50]
threshold1=[20]
point=[25]

In [126]:
def generate_time_range(df, entry_time, exit_time, interval_seconds=60):
    # Parsing entry and exit time so that we can add interval later
    entry_dt = datetime.strptime(entry_time, '%H:%M:%S')
    exit_dt = datetime.strptime(exit_time, '%H:%M:%S')

    # a list to save the time ranges to be created
    time_range = []

    # keep on adding intervals until current becomes equal or more than exit_time
    current_time = entry_dt
    while current_time <= exit_dt:
        if current_time.strftime('%H:%M:%S') in df['Timex'].values:
            time_range.append(current_time.strftime('%H:%M:%S'))
        else:
            for offset in range(-6,6):
                adjusted_time = (current_time + timedelta(seconds=offset)).time()
                if adjusted_time.strftime('%H:%M:%S') in df['Timex'].values:
                    time_range.append(adjusted_time.strftime('%H:%M:%S'))
                    break  # Exit the loop once a match is found
        current_time += timedelta(seconds=interval_seconds)

    return time_range 

In [127]:
def extract_numbers_from_pe_columns(df):
    # Get the list of column names ending with "PE"
    pe_columns = [col for col in df.columns if col.endswith('PEa')]

    # Extract numbers from column names
    numbers_from_pe_columns = []
    for col in pe_columns:
        numbers = re.findall(r'\d+', col)
        if numbers:
            numbers_from_pe_columns.extend(numbers)

    return numbers_from_pe_columns

In [128]:
def buy(row, strike_ce):
    if 'CE' in strike_ce:
        strike_ce = f'{strike_ce[:-2]}CEb'
    elif 'PE' in strike_ce:
        strike_ce = f'{strike_ce[:-2]}PEb'
    else:
        raise ValueError("strike_ce must contain either 'CE' or 'PE'")
    
    return row[strike_ce]


In [129]:
def sell(row, strike_ce):
    if 'CE' in strike_ce:
        
        strike_ce = f'{strike_ce[:-2]}CEa'
    elif 'PE' in strike_ce:
        # print(strike_ce)
        strike_ce = f'{strike_ce[:-2]}PEa'
    else:
        raise ValueError("strike_ce must contain either 'CE' or 'PE'")
    
    return row[strike_ce]


In [130]:
def closest_number(close_value, strikes_list):
    strikes_list = [int(x) for x in strikes_list]
    number = min(strikes_list, key=lambda x: abs(x - close_value))
    return number

In [131]:
def add_trade(trade_book, entry_time, strike, quantity, price,strike_type, description):
    trade_book.append({
        'entry_time': entry_time,
        'strike': strike,
        'quantity': quantity,
        'price': price,
        'strike_type':strike_type,
        'description': description
    })

In [132]:
def add_every_second(book , entry_time , strike_pe , strike_ce , pe_price ,ce_price,description,spot):
    book.append({'entry_time': entry_time,
                 'strike_pe' : strike_pe,
                 'strike_ce' : strike_ce,
                 'pe_price'  : pe_price,
                 'ce_price'  : ce_price,
                 'description': description ,
                 'spot':spot
                })

In [133]:
def slippage(book1 , diff):
    book1.append(diff)

In [134]:
def find_loss_time(df, loss_threshold=-50):
    # Initialize variables for PE and CE
    df.fillna(0, inplace=True)
    initial_pe_value = df.loc[0, 'pe_price']
    initial_ce_value = df.loc[0, 'ce_price']
    current_pe_description = df.loc[0, 'strike_pe']
    current_ce_description = df.loc[0, 'strike_ce']
    
    # Lists to hold the net changes
    net_changes_pe = []
    net_changes_ce = []
    pnl_pe = []
    pnl_ce = []

    for i in range(len(df)):
        # Check if the PE description has changed
        if df.loc[i, 'strike_pe'] != current_pe_description:
            # Update the initial net value and description
            initial_pe_value = df.loc[i, 'pe_price']
            current_pe_description = df.loc[i, 'strike_pe']
            pnl_pe.append((net_change_pe, df.loc[i, 'entry_time']))
        
        # Check if the CE description has changed
        if df.loc[i, 'strike_ce'] != current_ce_description:
            # Update the initial net value and description
            initial_ce_value = df.loc[i, 'ce_price']
            current_ce_description = df.loc[i, 'strike_ce']
            pnl_ce.append((net_change_ce, df.loc[i, 'entry_time']))
        
        # Calculate the net value and change for PE
        net_value_pe = df.loc[i, 'pe_price']
        if not pnl_pe:
            net_change_pe = initial_pe_value - net_value_pe
        else:
            net_change_pe = initial_pe_value - net_value_pe + pnl_pe[-1][0]
        net_changes_pe.append(net_change_pe)
        
        # Calculate the net value and change for CE
        net_value_ce = df.loc[i, 'ce_price']
        if not pnl_ce:
            net_change_ce = initial_ce_value - net_value_ce
        else:
            net_change_ce = initial_ce_value - net_value_ce + pnl_ce[-1][0]
        net_changes_ce.append(net_change_ce)
        
        # Check if the net change for PE meets the loss threshold
        if net_change_pe +net_change_ce <= loss_threshold:
            pnl_pe.append((net_change_pe, df.loc[i, 'entry_time']))
            pnl_ce.append((net_change_ce, df.loc[i, 'entry_time']))
            return df.loc[i, 'entry_time'], net_change_pe + net_change_ce
    
    return df.loc[i, 'entry_time'], (net_changes_pe[-1]+ net_changes_ce[-1])


In [135]:
def entry_strike(row, close_value, strike_interval):
    # Step 1: Find lower and upper strikes
    lower_strike = int((close_value // strike_interval) * strike_interval)
    upper_strike = lower_strike + strike_interval

    # Step 2: Define PE and CE for both strikes
    lowerPE = f"{lower_strike}PEb"
    lowerCE = f"{lower_strike}CEb"
    upperPE = f"{upper_strike}PEb"
    upperCE = f"{upper_strike}CEb"

    # Step 3: Fetch prices from the row
    lowerPE_price = row[lowerPE]
    lowerCE_price = row[lowerCE]
    upperPE_price = row[upperPE]
    upperCE_price = row[upperCE]

    # Step 4: Form all 4 combos (PE + CE)
    combos = [
        ((lowerPE, lowerPE_price), (lowerCE, lowerCE_price)),
        ((lowerPE, lowerPE_price), (upperCE, upperCE_price)),
        ((upperPE, upperPE_price), (lowerCE, lowerCE_price)),
        ((upperPE, upperPE_price), (upperCE, upperCE_price)),
    ]
    # print(combos)
    # Step 5: Find the combo with the closest prices between PE and CE
    best_combo = None
    smallest_diff = float('inf')

    for pe, ce in combos:
        if pe[1] is not None and ce[1] is not None:
            diff = abs(pe[1] - ce[1])
            if diff < smallest_diff:
                smallest_diff = diff
                best_combo = (pe, ce)

    # Step 6: Return the best combo - both strikes and their prices
    if best_combo:
        (pe_strike, pe_price), (ce_strike, ce_price) = best_combo
        # print(best_combo)
        return  pe_strike[:-1],pe_price, ce_strike[:-1],ce_price
        
    else:
        # If no valid combo was found (prices missing), return None
        # print('none')
        return None


In [136]:
#with percent
def calculation_pnl_for_threshold(trade_book,book,book1, df, entry_time, exit_time, current_leg, threshold, strike, new_strike, bought_leg_price, shift_price,i,j,thres1,pointt):
    if entry_time == exit_time:
        return None

    try:
        entry_index = df.index[df['Timex'] == entry_time][0]
    except IndexError:
        print(f'entry_time {entry_time} is not found in dataframe')
        return None
    i=0
    insideif = False
    check_price = shift_price
    current_price2 = shift_price

    entry_row = df.iloc[entry_index]
    spot = entry_row['Close']  
    
    if current_leg =='pe':
        add_every_second(book,entry_time,f'{new_strike}PE',None ,shift_price,None,'Threshold',spot)
    elif current_leg == 'ce': 
        add_every_second(book,entry_time,None , f'{new_strike}CE',None , shift_price,'threshold',spot)
        
    entryy = check_price
    for index, row in df.iloc[entry_index + 1:].iterrows():
        current_time = row['Timex']
        current_time_close=row['Close']
        if current_leg == 'pe':
            remaining_leg_price = buy(row,f'{new_strike}PE')#buy
            current_price = buy(row,f'{strike}')
            close_value = row['Close']
            
            if remaining_leg_price <= (entryy * (1 - j/100) ) :
                insideif=True
                check_price = remaining_leg_price*(1+ thres1/100)
                current_price2 = remaining_leg_price
                add_trade(trade_book,current_time,f'{new_strike}PE',0,0,'0',f'entry price was {entryy} current_price {remaining_leg_price}, new_threshold will be { remaining_leg_price*(1+ thres1/100)}')
                entryy = remaining_leg_price
            add_every_second(book,current_time,f'{new_strike}PE',None ,remaining_leg_price,None,'Threshold',current_time_close)
            
            if insideif and remaining_leg_price >= check_price:
                shift_price = current_price2
                ce_columns = [col for col in df.columns if col.endswith('CEb') and int(col[:-3]) >= int(close_value) - 25]
                rows = df.loc[index, ce_columns]
                rows = rows.apply(pd.to_numeric, errors='coerce')
                closest_column = rows.sub(remaining_leg_price).abs().idxmin()
                closest_column = closest_column[:-1]
                closest_strike = int(re.findall(r'\d+', closest_column)[0])
                new_ce_price = sell(row,closest_column)#sell
                strike_pe_buy = f'{new_strike - 1000}PE'
                pe_buy = sell(row,strike_pe_buy)#sell
                add_trade(trade_book , current_time,strike_pe_buy, -1 ,pe_buy ,'far_hedge_pe', 'threshold3 hit selling leg with 500 difference')
                add_trade(trade_book, current_time, closest_column, -1, new_ce_price,'main_ce', f'Threshold3 hit close_value:{close_value} hit at:{remaining_leg_price}')
                return {'threshold_hit_time': current_time,
                            'closest_column': closest_column,
                            'hit_price': current_price,
                            'price': new_ce_price,
                            'close value': close_value,
                            'p': remaining_leg_price,
                            's_price':shift_price,
                            'premium_recieved': new_ce_price + remaining_leg_price}
            if remaining_leg_price >= check_price * (1 + threshold / 100) and (not insideif) :
                shift_price = current_price2
                ce_columns = [col for col in df.columns if col.endswith('CEb') and int(col[:-3]) >= int(close_value) - 25]
                rows = df.loc[index, ce_columns]
                rows = rows.apply(pd.to_numeric, errors='coerce')
                closest_column = rows.sub(remaining_leg_price).abs().idxmin()
                closest_column = closest_column[:-1]
                closest_strike = int(re.findall(r'\d+', closest_column)[0])
                new_ce_price = sell(row,closest_column)#sell
                strike_pe_buy = f'{new_strike - 1000}PE'
                pe_buy = sell(row,strike_pe_buy)#sell
                # if insideif:
                #     print(current_time , 'point condition not considered')
                # else:
                #     print(current_time, 'point condition considered')
                add_trade(trade_book , current_time,strike_pe_buy, -1 ,pe_buy ,'far_hedge_pe', 'threshold hit selling leg with 500 difference')
                add_trade(trade_book, current_time, closest_column, -1, new_ce_price,'main_ce', f'Threshold hit close_value:{close_value} hit at:{remaining_leg_price}')
                return {'threshold_hit_time': current_time,
                            'closest_column': closest_column,
                            'hit_price': current_price,
                            'price': new_ce_price,
                            'close value': close_value,
                            'p': (remaining_leg_price+shift_price)/2,
                            's_price':shift_price,
                            'premium_recieved': new_ce_price + (remaining_leg_price+shift_price)/2}

        elif current_leg == 'ce':
            remaining_leg_price = buy(row,f'{new_strike}CE')#buy
            current_price = buy(row,f'{strike}')
            close_value = row['Close']
            
            if remaining_leg_price <= (entryy* (1 - j/100)):
                insideif = True
                check_price = remaining_leg_price*(1+ thres1/100)
                current_price2 = remaining_leg_price
                add_trade(trade_book,current_time,f'{new_strike}CE',0,0,'0',f'entry price was {entryy} current_price {remaining_leg_price}, new_threshold will be { remaining_leg_price*(1+ thres1/100)}')
                entryy = remaining_leg_price
            add_every_second(book,current_time,None , f'{new_strike}CE',None , remaining_leg_price,'threshold',current_time_close)
            if insideif and remaining_leg_price >= check_price:
                shift_price =  current_price2
                pe_columns = [col for col in df.columns if col.endswith('PEb') and int(col[:-3]) <= int(close_value) + 25]
                rows = df.loc[index, pe_columns]
                rows = rows.apply(pd.to_numeric, errors='coerce')
                closest_column = rows.sub(remaining_leg_price).abs().idxmin()
                closest_column = closest_column[:-1]
                closest_strike = int(re.findall(r'\d+', closest_column)[0])
                new_pe_price = sell(row,closest_column)#sell
                strike_ce_buy = f'{new_strike + 1000}CE'
                ce_buy = sell(row,strike_ce_buy)#sell
                add_trade(trade_book, current_time, strike_ce_buy, -1 , ce_buy,'far_hedge_ce', 'threshold3 buy leg with difference 500')
                add_trade(trade_book, current_time, closest_column, -1, new_pe_price,'main_pe', f'Threshold3 hit at {close_value} hit at:{remaining_leg_price}')
                return {'threshold_hit_time': current_time,
                            'closest_column': closest_column,
                            'hit_price': current_price,
                            'price': new_pe_price,
                            'close value': close_value,
                            'p': remaining_leg_price,
                            's_price':shift_price,
                            'premium_recieved': new_pe_price + remaining_leg_price}
            if remaining_leg_price >= check_price * (1 + threshold / 100) and (not insideif) :
                shift_price =  current_price2
                pe_columns = [col for col in df.columns if col.endswith('PEb') and int(col[:-3]) <= int(close_value) + 25]
                rows = df.loc[index, pe_columns]
                rows = rows.apply(pd.to_numeric, errors='coerce')
                closest_column = rows.sub(remaining_leg_price).abs().idxmin()
                closest_column = closest_column[:-1]
                closest_strike = int(re.findall(r'\d+', closest_column)[0])
                new_pe_price = sell(row,closest_column)#sell
                strike_ce_buy = f'{new_strike + 1000}CE'
                ce_buy = sell(row,strike_ce_buy) #sell
                # if insideif:
                #     print(current_time , 'point condition not considered')
                # else:
                #     print(current_time, 'point condition considered')
                add_trade(trade_book, current_time, strike_ce_buy, -1 , ce_buy,'far_hedge_ce', 'threshold buy leg with difference 500')
                add_trade(trade_book, current_time, closest_column, -1, new_pe_price,'main_pe', f'Threshold hit at {close_value} hit at:{remaining_leg_price}')
                return {'threshold_hit_time': current_time,
                            'closest_column': closest_column,
                            'hit_price': current_price,
                            'price': new_pe_price,
                            'close value': close_value,
                            'p': (remaining_leg_price+shift_price)/2,
                            's_price':shift_price,
                            'premium_recieved': new_pe_price + (remaining_leg_price+shift_price)/2}

        if current_time == exit_time:
            if current_leg == 'pe':
                add_trade(trade_book,current_time,f'{new_strike}PE',1,remaining_leg_price,'main_pe','Threshold not hit, buying remaining leg at exit')
                strike_pe_buy = f'{new_strike - 1000}PE'
                pe_buy = sell(row,strike_pe_buy)
                add_trade(trade_book , current_time,strike_pe_buy, -1 ,pe_buy ,'far_hedge_pe', 'threshold was not hit')
            if current_leg == 'ce':
                add_trade(trade_book,current_time,f'{new_strike}CE',1,remaining_leg_price,'main_ce','Threshold not hit, buying remaining leg at exit')
                strike_ce_buy = f'{new_strike + 1000}CE'
                ce_buy =sell(row,strike_ce_buy)
                add_trade(trade_book, current_time, strike_ce_buy, -1 , ce_buy,'far_hedge_ce', 'threshold was not hit')
            return None


In [137]:

def calculating_pnl_for_threshold_straddle(trade_book,book,book1, df, entry_time, exit_time, closest_column, new_strike, slr, isl, threshold,total_pnl, premium_recieved,target,percent,shift_price,j,thres1,pointt):
    try:
        entry_index = df.index[df['Timex'] == entry_time][0]
    except IndexError:
        print(f'entry_time {entry_time} is not found in dataframe')
        return None

    entry_row = df.iloc[entry_index]
    spot = entry_row['Close']  
    current_leg = 'both'

    if 'PE' in closest_column:
        strike_pe = closest_column
        strike_pe_buy = f'{int(strike_pe[:-2]) - percent}PE'
        # pe_buy = buy(entry_row,strike_pe_buy)#buy
        strike_ce = f'{new_strike}CE'
        strike_ce_buy = f'{new_strike + percent}CE'
        # lower_bound= int(strike_pe[:-2]) -(premium_recieved*(premium_percentage/100))
        # upper_bound = new_strike + (premium_recieved*(premium_percentage/100))
        entry_price_pe = sell(entry_row,closest_column)#sell
        entry_price_ce = shift_price
        # add_trade(trade_book , entry_time,strike_pe_buy, 1 ,pe_buy , 'buying with 100 points difference as we reenter')
        add_every_second(book,entry_time ,strike_pe,strike_ce ,entry_price_pe,entry_price_ce,'one',spot)
    elif 'CE' in closest_column:
        strike_ce = closest_column
        strike_ce_buy = f'{int(strike_ce[:-2]) + percent}CE'
        # ce_buy = entry_row[strike_ce_buy]
        strike_pe = f'{new_strike}PE'
        strike_pe_buy = f'{new_strike - percent}PE'
        # lower_bound = new_strike -(premium_recieved*(premium_percentage/100))
        # upper_bound = int(strike_ce[:-2]) - (premium_recieved*(premium_percentage/100))
        entry_price_pe = shift_price
        entry_price_ce = sell(entry_row,closest_column)#sell
        # add_trade(trade_book, entry_time, strike_ce_buy, 1 , ce_buy, 'buying with 100 points difference as we reenter')
        add_every_second(book,entry_time ,strike_pe,strike_ce ,entry_price_pe,entry_price_ce,'one',spot)
    # strike_pe_buy = f'{strike_pe[:-2] + 100}PE'
    # strike_ce_buy = f'{strike_ce[:-2] - 100}CE'
    pe_buy = buy(entry_row,strike_pe_buy)
    ce_buy = buy(entry_row,strike_ce_buy)
    add_trade(trade_book , entry_time,strike_pe_buy, 1 ,pe_buy ,'hedge_pe', 'buying with 100 points difference as we reenter')
    add_trade(trade_book, entry_time, strike_ce_buy, 1 , ce_buy,'hedge_ce', 'buying with 100 points difference as we reenter')
    spread_received = premium_recieved
    target=target
#     combined_stop_loss = spread_received * (1 + slr / 100)
    spread_received_hedge = pe_buy + ce_buy
    combined_stop_loss = spread_received - spread_received_hedge +slr
    # print(combined_stop_loss,spread_received,spread_received_hedge,entry_time)
    # current_pe_price = buy(entry_row,strike_pe)
    # current_ce_price = buy(entry_row,strike_ce)
    # current_ce_buy_price = sell(entry_row,strike_ce_buy)
    # current_pe_buy_price = sell(entry_row,strike_pe_buy)
    # current_spread_1= (current_pe_price + current_ce_price)-(current_ce_buy_price+current_pe_buy_price)
    # if current_spread_1 >= combined_stop_loss:
    #     combined_stop_loss = combined_stop_loss + 2.5
    ''' we will check the spread if the spread is less than 70% of the total spread than we buy and proceed
        if it is more than 70% percent than we  take slr of 5 ruppees'''
    # if (spread_received - spread_received_hedge)> 90:
    #     combined_stop_loss = spread_received - spread_received_hedge + 500
    i=0
    for index, row in df.iloc[entry_index + 1:].iterrows():
        current_close_value = row['Close']
        current_time = row['Timex']
        current_pe_price = buy(row,strike_pe)#buy
        current_ce_price = buy(row,strike_ce)#buy
        current_ce_buy_price = sell(row,strike_ce_buy)#sell
        current_pe_buy_price = sell(row,strike_pe_buy)#sell
        current_spread = current_pe_price + current_ce_price
        current_spread_1=current_spread-(current_ce_buy_price+current_pe_buy_price)
        diffce=current_ce_price-entry_price_ce
        diffpe=current_pe_price-entry_price_pe 
        add_every_second(book,current_time ,strike_pe,strike_ce ,current_pe_price,current_ce_price,'one',current_close_value)
        if current_spread_1 >= combined_stop_loss:
                # slippage(book1,(current_spread_1-combined_stop_loss))
                if current_pe_price > entry_price_pe and diffpe>diffce :
                    current_leg = 'ce'
                    first_pnl = entry_price_pe - current_pe_price
                    exit_price = current_pe_price
                    add_trade(trade_book, current_time, strike_pe, 1 ,exit_price,'main_pe',f'Individual stop_loss hit for PE spread = {current_spread_1} and combinedsl is {combined_stop_loss}')
                    add_trade(trade_book , current_time, strike_ce, 1 ,current_ce_price ,'main_ce',f'Buying CE , next we will shift as per Target {target} ,{spread_received}')
                    add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','selling the strike as isl is hit')
                    add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','selling the strike as isl is hit')
                    if current_ce_price >=target and False:
                        new_strike = strike_ce
                        new_entry_ce = current_ce_price
                    else:
                        ce_columns = [col for col in df.columns if col.endswith('CEb') and int(col[:-3]) >= int(current_close_value) - 25]
                        rows = df.loc[index, ce_columns].apply(pd.to_numeric, errors='coerce')
                        diff = rows - target
                        positive_diff = diff[diff > 0]
                        if positive_diff.empty:
                            new_strike = rows.sub(target).abs().idxmin()
                        else:
                            new_strike = rows.sub(target).abs().idxmin()
                        new_strike = new_strike[:-1]
                        new_entry_ce = sell(row,new_strike)# sell 
                        if current_ce_price > new_entry_ce:
                            new_strike = strike_ce
                            new_entry_ce = current_ce_price
                    new_strike = int(new_strike[:-2])
                    second_pnl = entry_price_ce - current_ce_price
                    shift_price = new_entry_ce
                    time_at_exit = row['Timex']
                    add_trade(trade_book, current_time, f'{int(new_strike)}CE', -1, new_entry_ce,'main_ce', f'Shifed CE to better premium, close_value:{current_close_value}')
                    strike_ce_buy = f'{new_strike + 1000}CE'
                    ce_buy = buy(row,strike_ce_buy)#buy
                    add_trade(trade_book, current_time, strike_ce_buy, 1 , ce_buy,'far_hedge_ce', 'buying with 500 points difference ')
                    i=i+1
                    result = calculation_pnl_for_threshold(trade_book,book,book1, df, current_time, exit_time, current_leg, threshold, strike_pe, new_strike, exit_price, shift_price,i,j,thres1,pointt)
                    if result is None:
                        # total_pnl += first_pnl + second_pnl + shift_price - df[df['Timex'] == exit_time][f'{new_strike}CE'].values[0]
                        break
                    else:
                        total_pnl += first_pnl + second_pnl
                        premium_recieved = result['premium_recieved']
                        entry_time = result['threshold_hit_time']
                        closest_column = result['closest_column']
                        shift_price = result['p']
                        total_pnl += calculating_pnl_for_threshold_straddle(trade_book,book,book1, df, entry_time, exit_time, closest_column, new_strike, slr, isl, threshold, total_pnl, premium_recieved,target,percent,shift_price,j,thres1,pointt)
                        break
    
                if current_ce_price >= entry_price_ce and diffce>diffpe:
                    current_leg = 'pe'
                    first_pnl = entry_price_ce - current_ce_price
                    exit_price = current_ce_price
                    add_trade(trade_book,current_time ,strike_ce , 1,current_ce_price,'main_ce', f'Individual stop_loss hit for CE, spread = {current_spread_1} and combinedsl is {combined_stop_loss}')
                    add_trade(trade_book,current_time,strike_pe,1,current_pe_price,'main_pe',f'Buying PE next we will shift as per target:{target} , {spread_received}')
                    add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','selling the strike as isl is hit')
                    add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','selling the strike as isl is hit')
                    if current_pe_price >= target and False:
                        new_strike = strike_pe
                        new_entry_pe = current_pe_price
                    else:
                        pe_columns = [col for col in df.columns if col.endswith('PEb') and int(col[:-3]) <= int(current_close_value) + 25]
                        rows_pe = df.loc[index, pe_columns].apply(pd.to_numeric, errors='coerce')
                        diff_pe = rows_pe - target
                        positive_diff_pe = diff_pe[diff_pe > 0]
                        if positive_diff_pe.empty:
                            new_strike = rows_pe.sub(target).abs().idxmin()
                            
                        else:
                            new_strike = rows_pe.sub(target).abs().idxmin()
                            
                        new_strike = new_strike[:-1]
                        new_entry_pe = sell(row,new_strike)# sell 
                        if current_pe_price > new_entry_pe:
                            new_strike = strike_pe
                            new_entry_pe = current_pe_price
                    new_strike = int(new_strike[:-2])
                    second_pnl = entry_price_pe - current_pe_price
                    shift_price = new_entry_pe
                    time_at_exit = row['Timex']
                    add_trade(trade_book, current_time, f'{int(new_strike)}PE',-1, new_entry_pe,'main_pe', f'Shifed PE to better premium, close_value:{current_close_value}')
                    strike_pe_buy = f'{new_strike - 1000}PE'
                    pe_buy = buy(row,strike_pe_buy)#buy
                    add_trade(trade_book , current_time,strike_pe_buy, 1 ,pe_buy ,'far_hedge_pe', 'buying with 500 points difference')
                    i=i+1
                    result = calculation_pnl_for_threshold(trade_book,book,book1, df, current_time, exit_time, current_leg, threshold, strike_ce, new_strike, exit_price, shift_price,i,j,thres1,pointt)
                    if result is None:
                        # total_pnl += first_pnl + second_pnl + entry_price_pe - df[df['Timex'] == exit_time][f'{new_strike}PE'].values[0]
                        break
                    else:
                        total_pnl += first_pnl + second_pnl
                        premium_recieved = result['premium_recieved']
                        entry_time = result['threshold_hit_time']
                        closest_column = result['closest_column']
                        shift_price = result['p']
                        total_pnl += calculating_pnl_for_threshold_straddle(trade_book,book,book1, df, entry_time, exit_time, closest_column, new_strike, slr, isl, threshold, total_pnl, premium_recieved,target,percent,shift_price,j,thres1,pointt)
                        break

        if current_time == exit_time:
            total_pnl += entry_price_ce - current_ce_price + entry_price_pe - current_pe_price
            add_trade(trade_book, current_time, strike_pe, 1, current_pe_price,'main_pe', 'Final Exit PE ')
            add_trade(trade_book, current_time, strike_ce, 1, current_ce_price,'main_ce', 'Final Exit CE ')
            add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','Final Exit PE ')
            add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','Final Exit CE ')
            break

    return total_pnl

In [138]:
def calculating_pnl_for_one_straddle(trade_book,book,book1, df, entry_time, exit_time, slr, isl, threshold,percent,j,thres1,pointt):
    # calculating_pnl_for_one_straddle(trade,book,book1, df, '12:46:00', '13:01:00',2.5,1, 30,100,40,20,50)
    try:
        entry_index = df.index[df['Timex'] == entry_time][0]
    except IndexError:
        print(f'entry_time {entry_time} is not found in dataframe')
        return None
    i=0
    isl =0
    pointt = 50

    
    
    entry_row = df.iloc[entry_index]
    close_value = entry_row['Close']   # spot 
    spot=close_value

    strike_pe,entry_price_pe,strike_ce,entry_price_ce = entry_strike(entry_row , close_value, 50)
  
    strike_pe_buy = f'{int(strike_pe[:-2]) - percent}PE'
    strike_ce_buy = f'{int(strike_ce[:-2]) + percent}CE'
    pe_buy = buy(entry_row,strike_pe_buy)#buy
    ce_buy = buy(entry_row,strike_ce_buy)#buy
    spread_received = entry_price_pe + entry_price_ce
    risk=spread_received
    #we will buy hedges of far 
    #we will keep checking the prices of unitl the csl real has not been hit current_sprea< combined spread
    # as then we will keep
    #that will also we our target to shift but quant 7 we will not be shifting remains with one leg only with this leg we hit threshold
#     combined_stop_loss = spread_received * (1 + slr / 100)
    spread_received_hedge = pe_buy + ce_buy
    combined_stop_loss = spread_received - spread_received_hedge +slr
    total_pnl = 0
    
    add_trade(trade_book, entry_time, strike_pe, -1, entry_price_pe,'main_pe', f'Initial Entry PE, close_value {close_value}')
    add_trade(trade_book, entry_time, strike_ce, -1, entry_price_ce,'main_ce', f'Initial Entry CE with spread received {spread_received}')
    add_trade(trade_book , entry_time,strike_pe_buy, 1 ,pe_buy ,'hedge_pe', 'buying with 100 points difference')
    add_trade(trade_book, entry_time, strike_ce_buy, 1 , ce_buy,'hedge_ce', 'buying with 100 points difference')
    add_every_second(book,entry_time ,strike_pe,strike_ce,entry_price_pe,entry_price_ce,'one',spot)
   
    target =max(entry_price_ce , entry_price_pe)

    #check if spread is less than csl previous
    # slippage(book1,0)
    
    current_leg = 'both'
    # current_strike = strike
    time_at_exit = exit_time
    current_spread = None
    result = None
    shift_price = None
    exit_price = None

    for index, row in df.iloc[entry_index + 1:].iterrows():
        
        current_close_value = row['Close']
        current_time = row['Timex']
        current_pe_price = buy(row,strike_pe)#buy
        current_ce_price = buy(row,strike_ce)#buy
        current_ce_buy_price = sell(row,strike_ce_buy)#sell
        current_pe_buy_price = sell(row,strike_pe_buy)#sell
        current_spread = current_pe_price + current_ce_price
        current_spread_1=current_spread-(current_ce_buy_price+current_pe_buy_price)
        diffce=current_ce_price-entry_price_ce
        diffpe=current_pe_price-entry_price_pe 
        add_every_second(book,current_time ,strike_pe ,strike_ce ,current_pe_price, current_ce_price,'one',current_close_value)
        
            
        if current_spread_1 >= combined_stop_loss:
                # slippage(book1,(current_spread_1-combined_stop_loss))
                if current_pe_price  > entry_price_pe and diffpe>diffce :
                    current_leg = 'ce'
                    first_pnl = entry_price_pe - current_pe_price
                    exit_price = current_pe_price
                    add_trade(trade_book, current_time, strike_pe, 1 ,exit_price,'main_pe',f'combined stop_loss hit for PE at spread = {current_spread_1} and combinedsl is {combined_stop_loss}')
                    add_trade(trade_book , current_time, strike_ce, 1 ,current_ce_price ,'main_ce',f'Buying CE , next we will shift as per Target {target}')
                    add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','selling the strike as isl is hit')
                    add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','selling the strike as isl is hit')
                    if current_ce_price >=target and False:
                        new_strike = strike_ce
                        new_entry_ce = current_ce_price
                    else:
                        ce_columns = [col for col in df.columns if col.endswith('CEa') and int(col[:-3]) >= int(current_close_value) - 25]
                        rows = df.loc[index, ce_columns].apply(pd.to_numeric, errors='coerce')
                        diff = rows - target #changes the way we find target
                        positive_diff = diff[diff > 0]
                        if positive_diff.empty:
                            new_strike = rows.sub(target).abs().idxmin()
                        else:
                            new_strike = rows.sub(target).abs().idxmin()
                        new_strike = new_strike[:-1]
                        new_entry_ce = sell(row,new_strike)# sell 
                        if current_ce_price > new_entry_ce:
                            new_strike = strike_ce
                            new_entry_ce = current_ce_price
                    new_strike = int(new_strike[:-2])
                    second_pnl = entry_price_ce - current_ce_price
                    shift_price = new_entry_ce
                    time_at_exit = row['Timex']
                    add_trade(trade_book, current_time, f'{int(new_strike)}CE', -1, new_entry_ce,'main_ce', f'Shifed CE to better premium, close_value:{current_close_value}')
                    strike_ce_buy = f'{new_strike + 1000}CE'
                    ce_buy = buy(row,strike_ce_buy) #buy
                    add_trade(trade_book, current_time, strike_ce_buy, 1 , ce_buy,'far_hedge_ce', 'buying with 1000 points difference ')
                    i=i+1
                    result = calculation_pnl_for_threshold(trade_book,book,book1, df, current_time, exit_time, current_leg, threshold, strike_pe, new_strike, exit_price, shift_price,i,j,thres1,pointt)
                    if result is None:
                        # total_pnl += first_pnl + second_pnl + shift_price - df[df['Timex'] == exit_time][f'{new_strike}CE'].values[0]
                        break
                    else:
                        total_pnl += first_pnl + second_pnl
                        premium_recieved = result['premium_recieved']
                        entry_time = result['threshold_hit_time']
                        closest_column = result['closest_column']
                        shift_price = result['p']
                        total_pnl += calculating_pnl_for_threshold_straddle(trade_book,book,book1, df, entry_time, exit_time, closest_column, new_strike, slr, isl, threshold, total_pnl, premium_recieved,target,percent,shift_price,j,thres1,pointt)
                        break
    
                if current_ce_price > entry_price_ce and diffce>diffpe:
                    current_leg = 'pe'
                    first_pnl = entry_price_ce - current_ce_price
                    exit_price = current_ce_price
                    add_trade(trade_book,current_time ,strike_ce , 1,current_ce_price,'main_ce', f'Individual stop_loss hit for CE, spread = {current_spread_1} and combinedsl is {combined_stop_loss}')
                    add_trade(trade_book,current_time,strike_pe,1,current_pe_price,'main_pe',f'Buying PE , next we will shift as per target:{target}')
                    add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','selling the strike as isl is hit')
                    add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','selling the strike as isl is hit')
                    if current_pe_price >= target and False:
                        new_strike = strike_pe
                        new_entry_pe = current_pe_price
                    else:
                        pe_columns = [col for col in df.columns if col.endswith('PEb') and int(col[:-3]) <= int(current_close_value) + 25]
                        rows_pe = df.loc[index, pe_columns].apply(pd.to_numeric, errors='coerce')
                        diff_pe = rows_pe - target
                        positive_diff_pe = diff_pe[diff_pe > 0]
                        if positive_diff_pe.empty:
                            new_strike = rows_pe.sub(target).abs().idxmin()
                            
                        else:
                            new_strike = rows_pe.sub(target).abs().idxmin()
                            
                        new_strike = new_strike[:-1]
                        new_entry_pe = sell(row,new_strike)# sell 
                        if current_pe_price > new_entry_pe:
                            new_strike = strike_pe
                            new_entry_pe = current_pe_price
                    new_strike = int(new_strike[:-2])
                    second_pnl = entry_price_pe - current_pe_price
                    shift_price = new_entry_pe
                    time_at_exit = row['Timex']
                    add_trade(trade_book, current_time, f'{int(new_strike)}PE',-1, new_entry_pe,'main_pe', f'Shifed PE to better premium, close_value:{current_close_value}')
                    strike_pe_buy = f'{new_strike - 1000}PE'
                    pe_buy = buy(row,strike_pe_buy)#buy
                    add_trade(trade_book , current_time,strike_pe_buy, 1 ,pe_buy ,'far_hedge_pe', 'buying with 1000 points difference')
                    i=i+1
                    result = calculation_pnl_for_threshold(trade_book,book,book1, df, current_time, exit_time, current_leg, threshold, strike_ce, new_strike, exit_price, shift_price,i,j,thres1,pointt)
                    if result is None:
                        # total_pnl += first_pnl + second_pnl + entry_price_pe - df[df['Timex'] == exit_time][f'{new_strike}PE'].values[0]
                        break
                    else:
                        total_pnl += first_pnl + second_pnl
                        premium_recieved = result['premium_recieved']
                        entry_time = result['threshold_hit_time']
                        closest_column = result['closest_column']
                        shift_price = result['p']
                        total_pnl += calculating_pnl_for_threshold_straddle(trade_book,book,book1, df, entry_time, exit_time, closest_column, new_strike, slr, isl, threshold, total_pnl, premium_recieved,target,percent,shift_price,j,thres1,pointt)
                        break
            
        if current_time == exit_time:
            total_pnl = spread_received - current_spread
            add_trade(trade_book, current_time, strike_pe, 1, current_pe_price,'main_pe', 'Final Exit PE as we did not met any conditions')
            add_trade(trade_book, current_time, strike_ce, 1, current_ce_price,'main_ce', 'Final Exit CE as we did not met any conditions')
            add_trade(trade_book , current_time , strike_pe_buy, -1, current_pe_buy_price,'hedge_pe','selling the strike as no condition is hit')
            add_trade(trade_book , current_time , strike_ce_buy, -1, current_ce_buy_price,'hedge_ce','selling the strike as no condition is hit')
            break
    
    return {
        'entry_time': entry_time,
        'total_pnl': total_pnl,
        'trade_book': trade_book,
        'risk':risk
    }

In [139]:
# df = pd.read_csv('n1/NIFTY_27032025_output.csv')


In [140]:
# df = pd.read_csv('NIFTY_20022025_output.csv')
# df

In [141]:
# trade = []
# book=[]
# book1=[]

# calculating_pnl_for_one_straddle(trade,book,book1, df, '12:46:00', '13:01:00',5,1, 30,100,40,20,50)

# trade =pd.DataFrame(trade)
# trade.to_excel('test_20022025.xlsx')
# grouped_data=trade.groupby('strike_type').apply(lambda group: -(group['quantity'] * group['price']).sum()).reset_index()


In [142]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Convert book to dataframe
# book_df = pd.DataFrame(book)

# # Make sure 'entry_time' is in datetime format
# book_df['entry_time'] = pd.to_datetime(book_df['entry_time'])

# # Sort by time just in case
# book_df = book_df.sort_values('entry_time').reset_index(drop=True)

# # Fill any missing PE or CE prices forward (sometimes prices are NaN if no trade happened)
# book_df['pe_price'] = book_df['pe_price'].fillna(method='ffill')
# book_df['ce_price'] = book_df['ce_price'].fillna(method='ffill')

# # Calculate per second PnL: (Initial PE+CE price) - (Current PE+CE price)
# initial_pe = book_df.loc[0, 'pe_price']
# initial_ce = book_df.loc[0, 'ce_price']

# # Create a new column for live PnL
# book_df['live_pnl'] = (initial_pe - book_df['pe_price'].fillna(0)) + (initial_ce - book_df['ce_price'].fillna(0))

# # Plotting
# plt.figure(figsize=(16, 6))
# plt.plot(book_df['entry_time'], book_df['live_pnl'], label='PnL over Time', color='blue')
# plt.xlabel('Time')
# plt.ylabel('PnL (in ₹)')
# plt.title('Live PnL per second')
# plt.grid(True)
# plt.legend()
# plt.show()


In [143]:
# book_df

In [144]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Convert book to dataframe
# book_df = pd.DataFrame(book)

# # Make sure 'entry_time' is in datetime format
# book_df['entry_time'] = pd.to_datetime(book_df['entry_time'])

# # Sort by time just in case
# book_df = book_df.sort_values('entry_time').reset_index(drop=True)

# # Fill any missing PE or CE prices forward (sometimes prices are NaN if no trade happened)
# book_df['pe_price'] = book_df['pe_price'].fillna(method='ffill')
# book_df['ce_price'] = book_df['ce_price'].fillna(method='ffill')

# # Calculate per second PnL: (Initial PE+CE price) - (Current PE+CE price)
# initial_pe = book_df.loc[0, 'pe_price']
# initial_ce = book_df.loc[0, 'ce_price']

# # Create a new column for live PnL
# book_df['live_pnl'] = (initial_pe - book_df['pe_price'].fillna(0)) + (initial_ce - book_df['ce_price'].fillna(0))



In [145]:
# fig = px.line(book_df, x='entry_time', y='live_pnl', title='Live PnL per second', labels={'entry_time': 'Time', 'live_pnl': 'PnL (in ₹)'})
# fig.update_traces(line=dict(color='blue'))

# # Update layout for better visualization
# fig.update_layout(
#     xaxis_title='Time', 
#     yaxis_title='PnL (in ₹)', 
   
#     width=1000,  # Width of the plot
#     height=700   # Height of the plot
# )

# fig.show()

In [146]:
# book_df

In [147]:
# import numpy as np
# import pandas as pd
# from scipy.stats import norm
# from scipy.optimize import brentq

# # Black-Scholes Pricing Functions
# def bs_call_price(S, K, T, r, sigma, q=0):
#     d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
#     d2 = d1 - sigma * np.sqrt(T)
#     return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# def bs_put_price(S, K, T, r, sigma, q=0):
#     d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
#     d2 = d1 - sigma * np.sqrt(T)
#     return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

# # Implied Volatility Function
# def implied_volatility(price, S, K, T, r, option_type, q=0):
#     if T <= 0 or price <= 0:
#         return np.nan

#     def objective_function(sigma):
#         if option_type == "CE":  # Call Option
#             return bs_call_price(S, K, T, r, sigma, q) - price
#         elif option_type == "PE":  # Put Option
#             return bs_put_price(S, K, T, r, sigma, q) - price
#         else:
#             return np.nan

#     try:
#         return brentq(objective_function, 1e-6, 5)  # Bounds for IV
#     except (ValueError, RuntimeError):
#         return np.nan

# # Constants
# R = 0.01  # Risk-free rate (1%)
# DIVIDEND = 0.0  # Dividend yield (0%)

# # Function to Calculate IV and BSM Prices
# def calculate_iv_bsm(row):
#     S = float(row['spot'])         # Spot price
#     # Check if the strike price columns are not NaN
#     if pd.notna(row['strike_ce']) and pd.notna(row['strike_pe']):
#         K_ce = float(row['strike_ce'].replace("CE", ""))  # Strike price for CE (remove 'CE')
#         K_pe = float(row['strike_pe'].replace("PE", ""))  # Strike price for PE (remove 'PE')
#     else:
#         return pd.Series([np.nan, np.nan, np.nan, np.nan])  # If NaN, return NaN for IV and BSM prices

#     T = 6 / 365                     # Time to expiry in years (6 days)
#     r = R                           # Risk-free rate
#     ce_price = row['ce_price']      # CE Option price
#     pe_price = row['pe_price']      # PE Option price
#     option_type_ce = "CE"           # CE Option
#     option_type_pe = "PE"           # PE Option
    
#     if pd.isna(ce_price) or ce_price <= 0 or T <= 0 or S <= 0:
#         ce_iv = np.nan
#         ce_bsm_price = np.nan
#     else:
#         ce_iv = implied_volatility(ce_price, S, K_ce, T, r, option_type_ce, DIVIDEND)
#         ce_bsm_price = bs_call_price(S, K_ce, T, r, ce_iv, DIVIDEND) if not np.isnan(ce_iv) else np.nan

#     if pd.isna(pe_price) or pe_price <= 0 or T <= 0 or S <= 0:
#         pe_iv = np.nan
#         pe_bsm_price = np.nan
#     else:
#         pe_iv = implied_volatility(pe_price, S, K_pe, T, r, option_type_pe, DIVIDEND)
#         pe_bsm_price = bs_put_price(S, K_pe, T, r, pe_iv, DIVIDEND) if not np.isnan(pe_iv) else np.nan

#     return pd.Series([ce_iv * 100 if not np.isnan(ce_iv) else np.nan, ce_bsm_price, pe_iv * 100 if not np.isnan(pe_iv) else np.nan, pe_bsm_price])

# # Apply the IV calculation to the DataFrame
# book_df[['ce_iv', 'ce_bsm_price', 'pe_iv', 'pe_bsm_price']] = book_df.apply(calculate_iv_bsm, axis=1)


In [148]:
# book_df

In [149]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # PnL vs CE_IV
# plt.figure(figsize=(14, 6))
# sns.scatterplot(x='ce_iv', y='live_pnl', data=book_df, color='blue', s=20)
# sns.lineplot(x='ce_iv', y='live_pnl', data=book_df, color='red')
# plt.title('PnL vs CE_IV (Seaborn)')
# plt.xlabel('CE IV')
# plt.ylabel('Live PnL')
# plt.grid(True)
# plt.show()

# # PnL vs PE_IV
# plt.figure(figsize=(14, 6))
# sns.scatterplot(x='pe_iv', y='live_pnl', data=book_df, color='green', s=20)
# sns.lineplot(x='pe_iv', y='live_pnl', data=book_df, color='red')
# plt.title('PnL vs PE_IV (Seaborn)')
# plt.xlabel('PE IV')
# plt.ylabel('Live PnL')
# plt.grid(True)
# plt.show()


In [150]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd

# # First, melt the dataframe to long format
# iv_df = book_df[['ce_iv', 'pe_iv', 'live_pnl']].copy()

# # Melt the DataFrame
# iv_melted = iv_df.melt(id_vars='live_pnl', value_vars=['ce_iv', 'pe_iv'],
#                        var_name='IV_Type', value_name='IV_Value')

# # Plot
# plt.figure(figsize=(14, 7))
# sns.scatterplot(x='IV_Value', y='live_pnl', hue='IV_Type', data=iv_melted, s=20)
# sns.lineplot(x='IV_Value', y='live_pnl', hue='IV_Type', data=iv_melted, legend=False)
# plt.title('PnL vs CE_IV and PE_IV (Combined View)')
# plt.xlabel('IV Value')
# plt.ylabel('Live PnL')
# plt.grid(True)
# plt.legend(title='IV Type')
# plt.show()


In [151]:
# import pandas as pd
# import plotly.graph_objects as go

# delta_pe = book_df['pe_price'].diff()
# delta_ce = book_df['ce_price'].diff()

# # Calculate gains and losses
# gain_pe = delta_pe.where(delta_pe > 0, 0)
# loss_pe = -delta_pe.where(delta_pe < 0, 0)

# gain_ce = delta_ce.where(delta_ce > 0, 0)
# loss_ce = -delta_ce.where(delta_ce < 0, 0)

# # Average gain and loss (14-period)
# average_gain_pe = gain_pe.rolling(window=14).mean()
# average_loss_pe = loss_pe.rolling(window=14).mean()

# average_gain_ce = gain_ce.rolling(window=14).mean()
# average_loss_ce = loss_ce.rolling(window=14).mean()

# # RSI Calculation for PE and CE
# rs_pe = average_gain_pe / average_loss_pe
# rsi_pe = 100 - (100 / (1 + rs_pe))

# rs_ce = average_gain_ce / average_loss_ce
# rsi_ce = 100 - (100 / (1 + rs_ce))

# # Combine PE and CE RSI
# book_df['rsi'] = (rsi_pe + rsi_ce) / 2

# # Create the figure with subplots for both PnL and RSI
# fig = go.Figure()

# # Plot PnL on the main y-axis
# fig.add_trace(go.Scatter(x=book_df['entry_time'], y=book_df['live_pnl'], mode='lines', name='Live PnL', line=dict(color='blue')))

# # Plot RSI on a secondary y-axis (right-side)
# fig.add_trace(go.Scatter(x=book_df['entry_time'], y=book_df['rsi'], mode='lines', name='RSI', line=dict(color='orange', dash='dot'), yaxis="y2"))

# # Add horizontal lines for overbought (70) and oversold (30) levels
# fig.add_hline(y=70, line=dict(color='red', dash='dash'), annotation_text='Overbought', annotation_position='top left', row=1, col=1)
# fig.add_hline(y=30, line=dict(color='green', dash='dash'), annotation_text='Oversold', annotation_position='bottom left', row=1, col=1)

# # Update layout for better visualization
# fig.update_layout(
#     title='Live PnL and RSI Analysis',
#     xaxis_title='Time', 
#     yaxis_title='PnL (in ₹)', 
    
#     width=1200,  # Width of the plot
#     height=800,  # Height of the plot
#     yaxis2=dict(
#         title='RSI',
#         overlaying='y',
#         side='right',
#         range=[0, 100]
#     ),
#     legend=dict(x=0.01, y=0.99)
# )

# # Show the plot
# fig.show()


In [152]:
# book_df

In [153]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd

# # Prepare the dataframe
# iv_df = book_df[['entry_time', 'ce_iv', 'pe_iv', 'live_pnl']].copy()
# iv_df['entry_time'] = pd.to_datetime(iv_df['entry_time'])  # Ensure entry_time is datetime

# # Melt the DataFrame
# iv_melted = iv_df.melt(id_vars=['entry_time', 'live_pnl'], value_vars=['ce_iv', 'pe_iv'],
#                        var_name='IV_Type', value_name='IV_Value')

# # Plot
# plt.figure(figsize=(16, 8))
# sns.scatterplot(x='IV_Value', y='live_pnl', hue='IV_Type', data=iv_melted, s=40)
# sns.lineplot(x='IV_Value', y='live_pnl', hue='IV_Type', data=iv_melted, legend=False)

# # Annotate time on each point
# for i in range(len(iv_melted)):
#     plt.text(iv_melted['IV_Value'].iloc[i], iv_melted['live_pnl'].iloc[i], 
#              iv_melted['entry_time'].iloc[i].strftime('%H:%M:%S'), 
#              fontsize=8, alpha=0.7)

# plt.title('PnL vs CE_IV and PE_IV (with Time Annotation)')
# plt.xlabel('IV Value')
# plt.ylabel('Live PnL')
# plt.grid(True)
# plt.legend(title='IV Type')
# plt.show()


In [154]:
# trade = pd.DataFrame(trade)
# trade

In [155]:
# trade.to_excel('tradebook_nifty0304_09_16_001.xlsx')

In [156]:
# print(trade.groupby('strike').sum('quantity'))
# print(sum(trade['quantity']*trade['price']*-1))

In [157]:
# entry_time = '10:53:25'
# entry_index = df.index[df['Time'] == entry_time][0]
# entry_row = df.iloc[entry_index]


In [158]:
# trade =pd.DataFrame(trade)
# trade
# df[df['Timex']=='09:26:36']['77400CE']

In [159]:
# trade.to_excel('NIFTY16012025_09_56_00_csl_15_isl_50_threshold_10_tradebook.xlsx')

In [160]:
# grouped_data=trade.groupby('strike_type').apply(lambda group: -(group['quantity'] * group['price']).sum()).reset_index()
# grouped_data

In [161]:
# with pd.ExcelWriter("tradebook05122024 with analysis.xlsx") as writer:
#     # Save the original DataFrame in one sheet
#     trade.to_excel(writer, sheet_name="Original_Data", index=False)
    
#     # Save the grouped analysis in another sheet named 'Analysis'
#     grouped_data.to_excel(writer, sheet_name="Analysis", index=False)

In [162]:
# trade.groupby('strike').sum('quantity')
# sum(trade['quantity']*trade['price'])

In [163]:
# print(x)

In [164]:
# sum(trade['quantity']*trade['price']*-1)

In [165]:
# i = int(input('entry file number:'))
# j = int(input(' exit file number:'))

import os

folder_path = 'n1'
file_list = sorted(os.listdir(folder_path))
files_to_process = file_list

files_to_process
# folder_path='n1/NIFTY_27032025_output.csv'
# file_list = sorted(os.listdir(folder_path))
# files_to_process = file_list[1:2:]
# files_to_process

['NIFTY_20022025_output.csv']

In [166]:
file_list

['NIFTY_20022025_output.csv']

In [167]:

# Function to process a single entry time
def process_entry_time(entry_time, df, file_name, csl_values, isl_values, threshold_values, percents, iterations,threshold1,point):
    pnl_row = {'file_name': file_name, 'entry_time': entry_time}
    for percent in percents:
        for csl in csl_values:
            for isl in isl_values:
                for threshold in threshold_values:
                    for iteration in iterations:
                        for thres1 in threshold1:
                            for pointt in point:
                                try:
                                    trade = []
                                    book = []
                                    book1=[]
                            # Assuming calculating_pnl_for_one_straddle handles one straddle calculation
                                    result = calculating_pnl_for_one_straddle(trade, book,book1, df, entry_time, '13:01:00', csl, isl, threshold, percent, iteration,thres1,pointt)
                                    trade = pd.DataFrame(trade)
                            
                            # Calculate PnL
                                    t = trade.eval('quantity * price').sum() * -1
                                    key = f'csl_{csl}_threshold_{threshold}_threshold1_{iteration}_threshold2_{thres1}'
                                    pnl_row[key] = t
                                except Exception as e:
                                    print(f"Error calculating PnL for {file_name} at {entry_time} with CSL={csl}, ISL={isl}, Threshold={threshold}, Percent={percent}, Iteration={iteration}: {e}")
    return pnl_row

# Function to process a single file, processing each entry time in parallel
def process_single_file(folder_path, file_name, csl_values, isl_values, threshold_values, percents, iterations,threshold1,point):
    summary_data = []
    print(f"Processing file: {file_name}")
    file_path = os.path.join(folder_path, file_name)

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        return summary_data

    try:
        time_range = generate_time_range(df, '12:46:00', '13:01:00')
    except Exception as e:
        print(f"Error generating time range for {file_name}: {e}")
        return summary_data

    # Process each entry time in parallel
    summary_data = Parallel(n_jobs=10)(
        delayed(process_entry_time)(entry_time, df, file_name, csl_values, isl_values, threshold_values, percents, iterations,threshold1,point)
        for entry_time in time_range
    )
    summary_dfs = pd.DataFrame(summary_data)
    summary_dfs.to_excel(f'{file_name}_test_12_46_13_01.xlsx')
    return summary_data

# Main function to process all files (sequentially in this case)
def process_files_to_dataframe_parallel(folder_path, files_to_process, csl_values, isl_values, threshold_values, percents, iterations,threshold1,point):
    all_data = []
    for file_name in files_to_process:
        # Process one file at a time, with parallel processing of entry times within the file
        file_data = process_single_file(folder_path, file_name, csl_values, isl_values, threshold_values, percents, iterations,threshold1,point)
        all_data.extend(file_data)

    summary_df = pd.DataFrame(all_data)
    
    return summary_df


In [168]:

# df_result = process_files_to_dataframe_parallel(folder_path, files_to_process, csl_values, isl_values, threshold_values,premium_percentages) 
summary_df = process_files_to_dataframe_parallel(folder_path, files_to_process, csl_values, isl_values, threshold_values,percents,iterations,threshold1,point)

Processing file: NIFTY_20022025_output.csv


In [169]:

def get_parameters_for_time(entry_time):
    # Define time ranges and corresponding parameters
    
    time_ranges = [
    (("09:16:00", "09:30:00"), {'csl': 2.5, 'threshold': 10, 'threshold1': 30, 'threshold2': 20, 'percent': 100}),
    (("09:31:00", "09:45:00"), {'csl': 5, 'threshold': 15, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("09:46:00", "10:00:00"), {'csl': 2.5, 'threshold': 15, 'threshold1': 30, 'threshold2': 40, 'percent': 100}),
    (("10:16:00", "10:30:00"), {'csl': 2.5, 'threshold': 10, 'threshold1': 30, 'threshold2': 20, 'percent': 100}),
    (("10:31:00", "10:45:00"), {'csl': 5, 'threshold': 15, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("10:46:00", "11:00:00"), {'csl': 5, 'threshold': 15, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("11:01:00", "11:15:00"), {'csl': 10, 'threshold': 20, 'threshold1': 30, 'threshold2': 40, 'percent': 100}),
    (("11:16:00", "11:30:00"), {'csl': 5, 'threshold': 20, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("11:31:00", "11:45:00"), {'csl': 7.5, 'threshold': 20, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("11:46:00", "12:00:00"), {'csl': 7.5, 'threshold': 20, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("12:01:00", "12:15:00"), {'csl': 5, 'threshold': 20, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("12:16:00", "12:30:00"), {'csl': 5, 'threshold': 20, 'threshold1': 30, 'threshold2': 40, 'percent': 100}),
    (("12:31:00", "12:45:00"), {'csl': 5, 'threshold': 20, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    (("12:46:00", "13:00:00"), {'csl': 5, 'threshold': 30, 'threshold1': 40, 'threshold2': 20, 'percent': 100}),
    ]










    # Convert entry_time string to datetime.time object
    entry_time_obj = datetime.strptime(entry_time, '%H:%M:%S').time()

    for (start_time, end_time), params in time_ranges:
        start_time_obj = datetime.strptime(start_time, '%H:%M:%S').time()
        end_time_obj = datetime.strptime(end_time, '%H:%M:%S').time()

        if start_time_obj <= entry_time_obj <= end_time_obj:
            return params

    return None  # No matching parameters


# Function to process a single entry time
def process_entry_time(entry_time, df, file_name):
    pnl_row = {'file_name': file_name, 'entry_time': entry_time}

    # Get specific parameters for this entry_time
    params = get_parameters_for_time(entry_time)
    if not params:
        print(f"No parameters found for {entry_time}. Skipping...")
        return pnl_row

    # Extract parameters
    csl = params['csl']
    isl = 1 
    threshold = params['threshold']
    percent = params['percent']
    threshold1 = params['threshold1']
    threshold2 = params['threshold2']

    try:
        trade = []
        book = []
        book1 = []
        # Replace with your actual PnL calculation logic
        result = calculating_pnl_for_one_straddle(trade, book,book1, df, entry_time, '13:01:00', 
                                                  csl, isl, threshold, percent, 
                                                  threshold1, threshold2, 25)  # trail fixed at 0
        trade = pd.DataFrame(trade)

        # Calculate PnL
        t =  trade.eval('quantity * price').sum() * -1
        key =  f'csl_{csl}_threshold_{threshold}_threshold1_{threshold1}_threshold2_{threshold2}'
        pnl_row[key] = t
    except Exception as e:
        print(f"Error calculating PnL for {file_name} at {entry_time}: {e}")

    return pnl_row


# Function to process a single file
def process_single_file(folder_path, file_name):
    summary_data = []
    print(f"Processing file: {file_name}")
    file_path = os.path.join(folder_path, file_name)

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        return summary_data

    try:
        time_range = generate_time_range(df, '12:46:00', '13:01:00',90)
    except Exception as e:
        print(f"Error generating time range for {file_name}: {e}")
        return summary_data

    # Process each entry time in parallel
    summary_data = Parallel(n_jobs=15)(
        delayed(process_entry_time)(entry_time, df, file_name) for entry_time in time_range
    )

    summary_dfs = pd.DataFrame(summary_data)
    summary_dfs.to_excel(f'{file_name}_backtest2_quant9c.xlsx', index=False)
    return summary_data


# Main function to process all files
def process_files_to_dataframe_parallel(folder_path, files_to_process):
    all_data = []
    for file_name in files_to_process:
        file_data = process_single_file(folder_path, file_name)
        all_data.extend(file_data)

    summary_df = pd.DataFrame(all_data)
    return summary_df



In [170]:
summary_df=process_files_to_dataframe_parallel(folder_path, files_to_process)

Processing file: NIFTY_20022025_output.csv


In [171]:
# path='sensex21012025 backtest.xlsx'
# summary_df.to_excel(path)

In [172]:
# summary_df

In [173]:
# initial_description='xxx'
# combination=[]
# for i , row in df.iterrows():
#     if row['descrition']!=initial_description:
#         combiantion.append(row['entry_time'],row['description']
#         initial_description = row['description']
